# This is a simple and naive approach for a big matrix decomposition

### 이 노트북에서는 instacart dataset의 사용자별 선호 상품을 SVD로 예측해본다.
### 다음과 같은 문제가 예상되고, 그에 대한 단순한 해결책을 제시한다.
    1. 매우 큰 차원의 데이터에 대한 decomposition을 어떻게 처리해야 하나?
    2. sparsity가 높은 행렬에 대한 SVD를 어떻게 처리해야 하나?

## data set: instacart(from kaggle)
    1. aisles.csv: 총 134개의 품목 섹션 데이터(대형 마트의 aisels = 통로, 물건 사이의 통로)
    2. order_products__prior.csv: 총 3243449개의 주문 정보. 주문자, 품목, 과거 주문 여부(0이면 신규주문)
    3. departments.csv: 코너 이름(냉동식품, 기타, 베이커리 등)
    4. products.csv: 총 상품 종류(49688개)
    5. orders.csv: 
    6. sample_submission.csv:
    7. order_products__train.csv: 학습용


## 1. simple EDA
    1. 기초통계량 및 분포 확인
    2. correlation between items

In [11]:
# import block
import pandas as pd
import numpy as np
import zipfile as zp
import os

In [28]:
# local path and file names
base_path = '/home/ssum/바탕화면/experiments/instacart/'
file_names = os.listdir(base_path)

In [29]:
file_names

['order_products__prior.csv.zip',
 'aisles.csv.zip',
 'departments.csv.zip',
 'products.csv.zip',
 'orders.csv.zip',
 'sample_submission.csv.zip',
 'order_products__train.csv.zip']

In [31]:
# order_products
zip_file = zp.ZipFile(base_path+file_names[0])
order_product__prior = pd.read_csv(zip_file.open('order_products__prior.csv'),sep=',')
order_product__prior

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [33]:
# order_products_train
zip_file = zp.ZipFile(base_path+file_names[6])
order_product__train = pd.read_csv(zip_file.open('order_products__train.csv'),sep=',')
order_product__train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [20]:
# aisels
zip_file = zp.ZipFile(base_path+file_names[1])
aisles = pd.read_csv(zip_file.open('aisles.csv'),sep=',')
aisles

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
...,...,...
129,130,hot cereal pancake mixes
130,131,dry pasta
131,132,beauty
132,133,muscles joints pain relief


In [32]:
# aisels
zip_file = zp.ZipFile(base_path+file_names[3])
products = pd.read_csv(zip_file.open('products.csv'),sep=',')
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8
